In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import gc
import math
from joblib import Parallel, delayed

from tqdm import tqdm
import time
import warnings

warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def get_psi(c):
    psi_res = pd.DataFrame()
    psi_dict={}
    # for c in tqdm(f_cols):
    try:
        t_train = x_train[c].fillna(-998)
        t_test = x_test[c].fillna(-998)
        #获取切分点
        bins=[]
        for i in np.arange(0,1.1,0.2):
            bins.append(t_train.quantile(i))
        bins=sorted(set(bins))
        bins[0]=-np.inf
        bins[-1]=np.inf
        #计算psi
        t_psi = pd.DataFrame()
        t_psi['train'] = pd.cut(t_train,bins).value_counts().sort_index()
        t_psi['test'] = pd.cut(t_test,bins).value_counts()
        t_psi.index=[str(x) for x in t_psi.index]
        t_psi.loc['总计',:] = t_psi.sum()
        t_psi['train_rate'] = t_psi['train']/t_psi.loc['总计','train']
        t_psi['test_rate'] = t_psi['test']/t_psi.loc['总计','test']
        t_psi['psi'] = (t_psi['test_rate']-t_psi['train_rate'])*(np.log(t_psi['test_rate'])-np.log(t_psi['train_rate']))
        t_psi.loc['总计','psi'] = t_psi['psi'].sum()
        t_psi.index.name=c
        #汇总
        t_res = pd.DataFrame([[c,t_psi.loc['总计','psi']]],
                             columns=['变量名','PSI'])
        psi_res = pd.concat([psi_res,t_res])
        psi_dict[c]=t_psi
        print(c,'done')
    except:
        print(c,'error')
    return psi_res #, psi_dict

In [3]:
def correlation(df, threshold=0.98):
    """
    特征相关性计算
    @param df: 
    @param threshold: 
    @return: 
    """
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colName = corr_matrix.columns[i]
                col_corr.add(colName)
    return col_corr

In [4]:
train_df = pd.read_hdf('../../input/train.h5')
test_df = pd.read_hdf('../../input/test.h5')

In [5]:
train_df = train_df[train_df['temperature'].notnull()]
train_df = train_df.fillna(method='bfill')
test_df = test_df.fillna(method='bfill')

In [6]:
train_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                    'indoorHum', 'indoorAtmo', 'temperature']
test_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                   'indoorHum', 'indoorAtmo']

In [7]:
train_df.drop(['year', 'sec'], axis=1, inplace=True)
test_df.drop(['year', 'sec'], axis=1, inplace=True)

In [8]:
train_df['indoorHum-outdoorHum'] = train_df['indoorHum'] - train_df['outdoorHum']
train_df['indoorAtmo-outdoorAtmo'] = train_df['indoorAtmo'] - train_df['outdoorAtmo']
# train_df['temperature-outdoorTemp'] = train_df['temperature'] - train_df['outdoorTemp']

test_df['indoorHum-outdoorHum'] = test_df['indoorHum'] - test_df['outdoorHum']
test_df['indoorAtmo-outdoorAtmo'] = test_df['indoorAtmo'] - test_df['outdoorAtmo']

In [9]:
train_df.loc[train_df['indoorAtmo-outdoorAtmo'] > 400, 'indoorAtmo-outdoorAtmo'] = 75
train_df.loc[train_df['indoorAtmo-outdoorAtmo'] < -400, 'indoorAtmo-outdoorAtmo'] = -93

test_df.loc[test_df['indoorAtmo-outdoorAtmo'] > 400, 'indoorAtmo-outdoorAtmo'] = 75
test_df.loc[test_df['indoorAtmo-outdoorAtmo'] < -400, 'indoorAtmo-outdoorAtmo'] = -93

In [10]:
train_df.loc[train_df['indoorHum-outdoorHum'] > 15, 'indoorHum-outdoorHum'] = 15
test_df.loc[test_df['indoorHum-outdoorHum'] > 15, 'indoorHum-outdoorHum'] = 15

In [11]:
train_count = train_df.shape[0]
y_train = train_df['temperature'].values - train_df['outdoorTemp'].values

In [12]:
train_df.shape

(24807, 13)

In [13]:
data_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

del train_df, test_df
gc.collect()

13

In [14]:
numerical_features = ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'indoorHum-outdoorHum', 'indoorAtmo-outdoorAtmo']
diff_features = ['{}_diff'.format(i) for i in numerical_features]
numerical_diff_features = numerical_features + diff_features

In [15]:
# for i in tqdm(numerical_features):
#     data_df['{}_diff'.format(i)] = data_df[i].diff()

In [16]:
data_df.fillna(method='bfill', inplace=True)

In [17]:
# 离散化
for f in numerical_features:
    data_df[f + '_20_bin'] = pd.cut(data_df[f], 20, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_50_bin'] = pd.cut(data_df[f], 50, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_100_bin'] = pd.cut(data_df[f], 100, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_200_bin'] = pd.cut(data_df[f], 200, duplicates='drop').apply(lambda x: x.left).astype(int)

In [18]:
for f1 in tqdm(['{}_20_bin'.format(i) for i in numerical_features] +
               ['{}_50_bin'.format(i) for i in numerical_features] +
               ['{}_100_bin'.format(i) for i in numerical_features] +
               ['{}_200_bin'.format(i) for i in numerical_features]):
    for f2 in numerical_features:
        tmp = data_df.groupby(f1, as_index=False)[f2].agg({
            '{}_{}_medi'.format(f1, f2): 'median',
            '{}_{}_mean'.format(f1, f2): 'mean',
            '{}_{}_max'.format(f1, f2): 'max',
            '{}_{}_min'.format(f1, f2): 'min',
            '{}_{}_sum'.format(f1, f2): 'sum',
            '{}_{}_std'.format(f1, f2): 'std',
            '{}_{}_skew'.format(f1, f2): 'skew'
        })
        data_df = data_df.merge(tmp, on=f1, how='left')
        del tmp
        gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [02:19<00:00,  4.98s/it]


In [19]:
train_df = data_df[:train_count].copy().reset_index(drop=True)
test_df = data_df[train_count:].copy().reset_index(drop=True)

del data_df
gc.collect()

20

In [20]:
train_time = train_df['time']
test_time = test_df['time']

In [21]:
drop_columns = ["time", "temperature"]

features = train_df[:1].drop(drop_columns, axis=1).columns
x_train = train_df[features]
x_test = test_df[features]

In [22]:
print(len(features))

1411


In [23]:
psi_res = Parallel(n_jobs=4)(delayed(get_psi)(c) for c in tqdm(features))
psi_df = pd.concat(psi_res)

100%|██████████████████████████████████████████████████████████████████████████████| 1411/1411 [00:19<00:00, 73.19it/s]


In [24]:
features = list(psi_df[psi_df['PSI'] > 0.2]['变量名'].values)

In [25]:
print(features)

['day', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorAtmo', 'outdoorTemp_20_bin', 'outdoorTemp_50_bin', 'outdoorTemp_100_bin', 'outdoorTemp_200_bin', 'outdoorHum_20_bin', 'outdoorHum_50_bin', 'outdoorHum_100_bin', 'outdoorHum_200_bin', 'outdoorAtmo_20_bin', 'outdoorAtmo_100_bin', 'outdoorAtmo_200_bin', 'indoorHum_20_bin', 'indoorAtmo_20_bin', 'indoorAtmo_50_bin', 'indoorAtmo_100_bin', 'indoorAtmo_200_bin', 'outdoorTemp_20_bin_outdoorTemp_medi', 'outdoorTemp_20_bin_outdoorTemp_mean', 'outdoorTemp_20_bin_outdoorTemp_max', 'outdoorTemp_20_bin_outdoorTemp_min', 'outdoorTemp_20_bin_outdoorTemp_sum', 'outdoorTemp_20_bin_outdoorTemp_std', 'outdoorTemp_20_bin_outdoorHum_medi', 'outdoorTemp_20_bin_outdoorHum_mean', 'outdoorTemp_20_bin_outdoorHum_max', 'outdoorTemp_20_bin_outdoorHum_min', 'outdoorTemp_20_bin_outdoorHum_sum', 'outdoorTemp_20_bin_outdoorHum_std', 'outdoorTemp_20_bin_outdoorHum_skew', 'outdoorTemp_20_bin_outdoorAtmo_medi', 'outdoorTemp_20_bin_outdoorAtmo_mean', 'outdoorTemp_2

In [26]:
print(len(features))

580


In [27]:
x_train.drop(features, axis=1, inplace=True)
x_test.drop(features, axis=1, inplace=True)
gc.collect()

682

In [28]:
x_train.shape

(24807, 831)

In [29]:
x_train.head(10).append(x_train.tail(10))

month  hour  min  indoorHum  indoorHum-outdoorHum  \
0          3     1    0       80.0                  -5.0   
1          3     1    1       80.0                  -4.0   
2          3     1    2       80.0                  -4.0   
3          3     1    3       80.0                  -5.0   
4          3     1    4       80.0                  -5.0   
5          3     1    5       80.0                  -5.0   
6          3     1    6       80.0                  -5.0   
7          3     1    7       80.0                  -5.0   
8          3     1    8       80.0                  -5.0   
9          3     1    9       80.0                  -5.0   
24797      4     0   50       75.0                  -3.0   
24798      4     0   51       75.0                  -3.0   
24799      4     0   52       75.0                  -3.0   
24800      4     0   53       75.0                  -2.0   
24801      4     0   54       75.0                  -3.0   
24802      4     0   55       75.0                  -3.0   
24803      4     0   56       75.0                  -2.0   
24804      4     0   57       75.0                  -2.0   
24805      4     0   58       75.0                  -2.0   
24806      4     0   59       75.0                  -2.0   

       indoorAtmo-outdoorAtmo  outdoorAtmo_50_bin  indoorHum_50_bin  \
0                        -0.7                 986                79   
1                        -0.4                 986                79   
2                         0.1                 986                79   
3                        -0.6                 986                79   
4                        -0.1                 986                79   
5                        -0.2                 986                79   
6                       -93.0                 986                79   
7                        -0.8                 986                79   
8                        -0.2                 986                79   
9                        -0.6                 986                79   
24797                    -1.6                 973                73   
24798                    -1.8                 973                73   
24799                    -1.3                 973                73   
24800                    -1.6                 973                73   
24801                    -1.3                 973                73   
24802                    -1.6                 973                73   
24803                    -0.9                 973                73   
24804                    -0.9                 973                73   
24805                    -0.8                 973                73   
24806                    -0.5                 973                73   

       indoorHum_100_bin  indoorHum_200_bin  indoorHum-outdoorHum_20_bin  \
0                     79                 79                           -5   
1                     79                 79                           -4   
2                     79                 79                           -4   
3                     79                 79                           -5   
4                     79                 79                           -5   
5                     79                 79                           -5   
6                     79                 79                           -5   
7                     79                 79                           -5   
8                     79                 79                           -5   
9                     79                 79                           -5   
24797                 74                 74                           -4   
24798                 74                 74                           -4   
24799                 74                 74                           -4   
24800                 74                 74                           -3   
24801                 74                 74                           -4   
24802                 74              

In [30]:
x_test.head(10).append(x_test.tail(10))

month  hour  min  indoorHum  indoorHum-outdoorHum  \
0        4     1    0       88.0                  -3.0   
1        4     1   30       88.0                  -3.0   
2        4     2    0       89.0                  -3.0   
3        4     2   30       89.0                  -3.0   
4        4     3    0       88.0                  -3.0   
5        4     3   30       88.0                  -3.0   
6        4     4    0       89.0                  -3.0   
7        4     4   30       89.0                  -4.0   
8        4     5    0       88.0                  -4.0   
9        4     5   30       88.0                  -3.0   
396      4    20   27       59.0                  -1.0   
397      4    20   57       61.0                   1.0   
398      4    21   27       62.0                   0.0   
399      4    21   57       64.0                  -1.0   
400      4    22   27       65.0                  -2.0   
401      4    22   57       66.0                   0.0   
402      4    23   28       65.0                  -1.0   
403      4    23   58       65.0                   0.0   
404      4     0   28       48.0                   4.0   
405      4     0   57       47.0                   4.0   

     indoorAtmo-outdoorAtmo  outdoorAtmo_50_bin  indoorHum_50_bin  \
0                       0.3                 986                87   
1                       0.2                 986                87   
2                       0.3                 986                88   
3                       0.6                 986                88   
4                       0.0                 986                87   
5                      -2.0                 986                87   
6                      -1.4                 986                88   
7                      -0.8                 986                88   
8                     -93.0                 986                87   
9                      -1.8                 986                87   
396                     0.6                 973                58   
397                     0.2                 973                60   
398                     0.0                 973                61   
399                     0.5                 973                63   
400                     0.4                 973                64   
401                    -2.2                 973                65   
402                    -2.1                 973                64   
403                     0.7                 973                64   
404                     0.9                 973                47   
405                     1.1                 973                46   

     indoorHum_100_bin  indoorHum_200_bin  indoorHum-outdoorHum_20_bin  \
0                   87                 87                           -4   
1                   87                 87                           -4   
2                   88                 88                           -4   
3                   88                 88                           -4   
4                   87                 87                           -4   
5                   87                 87                           -4   
6                   88                 88                           -4   
7                   88                 88                           -4   
8                   87                 87                           -4   
9                   87                 87                           -4   
396                 58                 58                           -1   
397                 60                 60                            0   
398                 61                 61                            0   
399                 63                 63                           -1   
400                 64                 64                           -3   
401                 65                 65                            0   
402                 64                 64                           -1   
403 

In [31]:
col_corr = correlation(x_train, 0.90)
print(len(col_corr))

557


In [32]:
x_train.drop(list(col_corr), axis=1, inplace=True)
x_test.drop(list(col_corr), axis=1, inplace=True)

In [33]:
x_train['time'] = train_time
x_test['time'] = test_time

In [34]:
print(x_train.columns.to_list())

['month', 'hour', 'min', 'indoorHum', 'indoorHum-outdoorHum', 'indoorAtmo-outdoorAtmo', 'outdoorAtmo_50_bin', 'outdoorTemp_20_bin_outdoorTemp_skew', 'outdoorTemp_20_bin_outdoorAtmo_std', 'outdoorTemp_20_bin_outdoorAtmo_skew', 'outdoorTemp_20_bin_indoorAtmo_std', 'outdoorTemp_20_bin_indoorAtmo-outdoorAtmo_mean', 'outdoorTemp_20_bin_indoorAtmo-outdoorAtmo_max', 'outdoorTemp_20_bin_indoorAtmo-outdoorAtmo_min', 'outdoorHum_20_bin_outdoorTemp_sum', 'outdoorHum_20_bin_outdoorTemp_std', 'outdoorHum_20_bin_outdoorTemp_skew', 'outdoorHum_20_bin_outdoorHum_std', 'outdoorHum_20_bin_outdoorHum_skew', 'outdoorHum_20_bin_outdoorAtmo_min', 'outdoorHum_20_bin_outdoorAtmo_std', 'outdoorHum_20_bin_indoorHum_skew', 'outdoorHum_20_bin_indoorAtmo_max', 'outdoorHum_20_bin_indoorAtmo_min', 'outdoorHum_20_bin_indoorAtmo_std', 'outdoorHum_20_bin_indoorAtmo_skew', 'outdoorHum_20_bin_indoorHum-outdoorHum_sum', 'outdoorHum_20_bin_indoorHum-outdoorHum_std', 'outdoorHum_20_bin_indoorHum-outdoorHum_skew', 'outdoorHu

In [35]:
to_drop = ['month', 'hour', 'min', 'indoorHum', 'indoorHum-outdoorHum', 'indoorAtmo-outdoorAtmo']

In [36]:
data_df = pd.concat([x_train, x_test])
data_df.to_hdf('../../input/features/agg.h5', 'df')